In [12]:
%cd ~/big_vision
import jax
import ml_collections
import big_vision.models.proj.image_text.two_towers as model_mod

/home/austinwang/big_vision


In [7]:
VARIANT, RES = 'B/16', 224
CKPT, TXTVARIANT, EMBDIM, SEQLEN, VOCAB = 'siglip/webli_en_b16_224_63724782.npz', 'B', 768, 64, 32_000

In [8]:
model_cfg = ml_collections.ConfigDict()
model_cfg.image_model = 'vit'  # TODO(lbeyer): remove later, default
model_cfg.text_model = 'proj.image_text.text_transformer'  # TODO(lbeyer): remove later, default
model_cfg.image = dict(variant=VARIANT, pool_type='map')
model_cfg.text = dict(variant=TXTVARIANT, vocab_size=VOCAB)
model_cfg.out_dim = (None, EMBDIM)  # (image_out_dim, text_out_dim)
model_cfg.bias_init = -10.0
model_cfg.temperature_init = 10.0

model = model_mod.Model(**model_cfg)

# Using `init_params` is slower but will lead to `load` below performing sanity-checks.
# init_params = jax.jit(model.init, backend="cpu")(jax.random.PRNGKey(42), jnp.zeros([1, RES, RES, 3], jnp.float32), jnp.zeros([1, SEQLEN], jnp.int32))['params']
init_params = None  # Faster but bypasses loading sanity-checks.

params = model_mod.load(init_params, f'/mnt/vlm-pd/ckpts/{CKPT}', model_cfg)

In [13]:
jax.tree_util.tree_map(lambda x: x.shape, params)

{'b': (1,),
 'img': {'MAPHead_0': {'LayerNorm_0': {'bias': (768,), 'scale': (768,)},
   'MlpBlock_0': {'Dense_0': {'bias': (3072,), 'kernel': (768, 3072)},
    'Dense_1': {'bias': (768,), 'kernel': (3072, 768)}},
   'MultiHeadDotProductAttention_0': {'key': {'bias': (12, 64),
     'kernel': (768, 12, 64)},
    'out': {'bias': (768,), 'kernel': (12, 64, 768)},
    'query': {'bias': (12, 64), 'kernel': (768, 12, 64)},
    'value': {'bias': (12, 64), 'kernel': (768, 12, 64)}},
   'probe': (1, 1, 768)},
  'Transformer': {'encoder_norm': {'bias': (768,), 'scale': (768,)},
   'encoderblock_0': {'LayerNorm_0': {'bias': (768,), 'scale': (768,)},
    'LayerNorm_1': {'bias': (768,), 'scale': (768,)},
    'MlpBlock_0': {'Dense_0': {'bias': (3072,), 'kernel': (768, 3072)},
     'Dense_1': {'bias': (768,), 'kernel': (3072, 768)}},
    'MultiHeadDotProductAttention_0': {'key': {'bias': (12, 64),
      'kernel': (768, 12, 64)},
     'out': {'bias': (768,), 'kernel': (12, 64, 768)},
     'query': {'bi